In [24]:
import boto3
import pandas as pd
from io import StringIO

In [26]:
# create s3 client
s3 = boto3.client('s3')

# specify the bucket & folder you'd like to interact with
# we will be using the jobs team as an example
name = "ds-education-la"
file = "raw/NYC_School_Data.csv"

# using these parameters, we get our "gsearch_jobs.csv" object from s3
single_object = s3.get_object(Bucket=name, Key=file)

In [27]:
single_object

{'ResponseMetadata': {'RequestId': 'V645GZE20JG97Q7H',
  'HostId': 'mxkivlDEjQ89SXifly3iR6wdDxePIbqqslv2M/MbjLeJiqGsK5payEGoQ1QHXHMES5ZIOcSR8rSeAcHpXPqt/xqITtGGZD4f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mxkivlDEjQ89SXifly3iR6wdDxePIbqqslv2M/MbjLeJiqGsK5payEGoQ1QHXHMES5ZIOcSR8rSeAcHpXPqt/xqITtGGZD4f',
   'x-amz-request-id': 'V645GZE20JG97Q7H',
   'date': 'Thu, 30 May 2024 02:17:30 GMT',
   'last-modified': 'Thu, 30 May 2024 02:15:34 GMT',
   'etag': '"aca2fbdf5ce6124da089e43192da55c5"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'server': 'AmazonS3',
   'content-length': '325088'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2024, 5, 30, 2, 15, 34, tzinfo=tzutc()),
 'ContentLength': 325088,
 'ETag': '"aca2fbdf5ce6124da089e43192da55c5"',
 'ContentType': 'text/csv',
 'ServerSideEncryption': 'AES256',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x

In [28]:
# then we subsequently save it into a pandas dataframe to prepare it for transformations
df = pd.read_csv(single_object['Body'])

In [29]:
# let's confirm that this worked by viewing the first 5 rows
df.head()

,District,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,1,3,2023,Econ Disadv,313,442,101,32.3,90,28.8,86,27.5,36,11.5,122,39.0
1,1,3,2023,Not Econ Disadv,207,465,20,9.7,24,11.6,60,29.0,103,49.8,163,78.7
2,1,4,2023,Econ Disadv,306,448,65,21.2,102,33.3,79,25.8,60,19.6,139,45.4
3,1,4,2023,Not Econ Disadv,191,473,7,3.7,28,14.7,39,20.4,117,61.3,156,81.7
4,1,5,2023,Econ Disadv,335,445,96,28.7,92,27.5,101,30.1,46,13.7,147,43.9


## Transform


In [30]:
# let's drop all columns except for Grade, Year, %level 1, %level 2, %level 3, %level 4

important_cols = ['Grade', 'Year', '% Level 1', '% Level 2', '% Level 3', '% Level 4']

focused_df = df[important_cols]

focused_df.head()

,Grade,Year,% Level 1,% Level 2,% Level 3,% Level 4
0,3,2023,32.3,28.8,27.5,11.5
1,3,2023,9.7,11.6,29.0,49.8
2,4,2023,21.2,33.3,25.8,19.6
3,4,2023,3.7,14.7,20.4,61.3
4,5,2023,28.7,27.5,30.1,13.7


In [31]:
# let's take a look at the null values in this dataframe

focused_df.isna().sum()

Grade        0
Year         0
% Level 1    0
% Level 2    0
% Level 3    0
% Level 4    0
dtype: int64

In [32]:
focused_df.shape

(4032, 6)

In [33]:
# drop all missing values

dropped_df = focused_df.dropna()

In [34]:
dropped_df.shape

(4032, 6)

In [35]:
dropped_df.head()

,Grade,Year,% Level 1,% Level 2,% Level 3,% Level 4
0,3,2023,32.3,28.8,27.5,11.5
1,3,2023,9.7,11.6,29.0,49.8
2,4,2023,21.2,33.3,25.8,19.6
3,4,2023,3.7,14.7,20.4,61.3
4,5,2023,28.7,27.5,30.1,13.7


The dataset is pretty complete and we are ready to proceed.

 ## Load


In [42]:
# create a buffer object (temp place to hold data)
csv_buffer = StringIO()

# open the file in binary format, and save into the var 'data'
dropped_df.to_csv(csv_buffer, index=False)

### ADDING OBJECTS TO A BUCKET ###
response = s3.put_object(
    Body=csv_buffer.getvalue(),
    Bucket=name,
    Key= "NYC_School_Data.xlsx"
)